# AI Agent

An AI agent is an intelligent system that recieves a high-level goal from a user, and autonomously plans, decides, and executes a sequence of actions by using external tools, APIs, or knowledgeable sources - all while maintaining context, reasoning over multiple steps, adapting to new information, and optimizing for the intended outcome.

The characteristcs of an AI agent are:
 - **Goal Driven** : You tell agent what you want , not how to do it
 - **Autonomous planning** - Agent breaks down the problem and sequences task on its own
 - **Tool Using** - Agants calls APIs, calculators, search tools, etc.
 - **Context-Aware** - Maintains memory across steps to inform future actions
 - **Adaptive** - Rethinks plan when things change (e.g. API fails, no data)

In [1]:
import os
import requests

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

In [3]:
search_tool = DuckDuckGoSearchRun()

In [5]:
results = search_tool.invoke('Top News in Pakistan Today')
results

'Stay on top of Pakistan latest developments on the ground with Al Jazeera\'s fact-based news, exclusive video footage, photos and updated maps. 📌 Meanwhile, BJP spokesperson, Guru Prakash, paints the Centre\'s decision as a "political and moral milestone" to overturn years of the Congress\'s "lackadaisical attitude towards social justice".. The ruling party, in fact, is all geared up to counter any projection that this is a vindication of the Opposition\'s stand. The BJP plans to invoke the Congress\'s lack of action on ... Pakistan\'s government claimed late Tuesday night that it had "credible intelligence" that India was planning to take military action within hours in retaliation for the April 22 terrorist attack ... The document, a National Intelligence Estimate (NIE), studied the India-Pakistan dynamic and offered one conclusion: if war erupts, it will likely start with something like Kashmir, and Pakistan ... ARY NEWS brings you 24/7 Live Streaming, Headlines, Bulletins, Talk Sh

In [6]:
llm = ChatOpenAI()

In [7]:
llm.invoke('Hi')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BSm2R6kLoBEmmm141CABBixs7E5m0', 'finish_reason': 'stop', 'logprobs': None}, id='run-64ab6549-6d10-472f-983e-55e703faf339-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [9]:
# step 2: Pull th ReAct prompt from Lnagchain hub
prompt = hub.pull('hwchase17/react')

d:\Learning\GenAI\Dev\Langchain\.venv\Lib\site-packages\langsmith\client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
# Step 3: Create the ReAct agent manually with the pulled prompt

agent = create_react_agent(llm=llm,
                           tools=[search_tool],
                           prompt=prompt)

In [11]:
# Step 4 : Wrap it with Agent Executor
agent_executor = AgentExecutor(agent=agent,
                               tools=[search_tool],
                               verbose=True)

In [15]:
# Step 5: Invoke
response = agent_executor.invoke({"input" : " Give a preview for PSL Quetta Gladdiator and Multan Sultans"})
print(response)



> Entering new AgentExecutor chain...
Let's gather information about the upcoming match between PSL teams Quetta Gladiators and Multan Sultans.
Action: duckduckgo_search
Action Input: PSL Quetta Gladiators vs Multan Sultans previewGaddafi Stadium in Lahore will welcome Quetta Gladiators and Multan Sultans, who will square off tonight in Match 18 of the Pakistan Super League (PSL) 2025. Both sides are at very different ends ... Quetta Gladiators vs Multan Sultans prediction The Quetta Gladiators come into this one on the back of back-to-back victories. The most recent of which saw them defeat Peshawar Zalmi by 64 runs in Quetta the other day, scoring 178-7 in 20 overs with contributions from all of the top order before skittling Peshawar for just 114 thanks to a five ... Live on Quetta Gladiators Official Social Media | DRS - De Review Show | PSL 2025 | Match Preview - Quetta Gladiators vs Multan SultansWelcome to a special L... The Gladiators will present a difficult challenge to the

In [16]:
response['output']

'Quetta Gladiators have a head-to-head record of 4 wins against Multan Sultans who have 9 wins. Quetta Gladiators are in good form with back-to-back victories and have key players like Saud Shakeel and Finn Allen. Multan Sultans are struggling this season and recently suffered a big loss to Quetta Gladiators with a record-breaking 10-wicket victory.'



# ReAct

ReAct is a design pattern used in AI agents that stands for Reasoning + Acting. It combines:,

1. **Reasoning**: The agent thinks about what to do by:
   - Analyzing the current situation
   - Breaking down complex tasks
   - Planning next steps
   - Learning from previous actions

2. **Acting**: The agent takes concrete actions by:
   - Using available tools (like search, APIs, calculators)
   - Executing planned steps
   - Gathering new information

The ReAct pattern follows this loop:
1. **Thought**: Agent reasons about the current state and what to do next
2. **Action**: Agent selects and uses an appropriate tool
3. **Observation**: Agent processes the results of the action
4. **Repeat**: Agent continues this cycle until the goal is achieved
   
In the code above, we implemented ReAct using:

- `create_react_agent()`: Creates an agent with the ReAct prompt pattern
- `AgentExecutor`: Manages the execution loop of thinking and acting
- Tools like `DuckDuckGoSearchRun`: Provides capabilities for actions


# Agent and AgentExecutor

`AgentExecutor` orchestrate the entire loop:
 - Sends the input and previous messages to the agent
 - Gets the next `action` from agent
 - Executes that tool woth provided input
 - Adds the tools's `observation` basck into the history
 - Loops again with updated history until the agent says `Final Answer`